In [1]:
import os

from pyfrac.utils.sofiutils import *

In [2]:
def combineSOFI_strings(default_strs, monitor_strs,  t_str, mod_strs, src_str):

    pre_blurb = '''#-----------------------------------------------------------------
#      JSON PARAMETER FILE FOR SOFI3D
#-----------------------------------------------------------------
# description: example of json input file
# description/name of the model: homogeneous full space (hh.c)
#
'''

    string_list = [pre_blurb, '{', 
                   default_strs,
                   monitor_strs, 
                   t_str,
                   mod_strs,
                   src_str,
                   '}']
    SOFIstring = '\n \n'.join(string_list)
    return SOFIstring 

In [3]:
def get_default_strings():
    
    chkpt_str = '''"Checkpoints" : "comment",
    "CHECKPTREAD" : "0",
    "CHECKPTWRITE" : "0",
    "CHECKPT_FILE" : "tmp/checkpoint_sofi3D",'''
    
    dom_decomp_str = '''"Domain Decomposition" : "comment",
    "NPROCX" : "2",
    "NPROCY" : "2",
    "NPROCZ" : "2",'''
    
    fdorder_str = '''"FD order" : "comment",
    "FDORDER" : "4",
    "FDORDER_TIME" : "2",
    "FDCOEFF" : "2",
    "fdcoeff values: Taylor=1, Holberg=2" : "comment",'''
    
    q_str = '''"Q-approximation" : "comment",
    "L" : "0",
    "FREF" : "5.0",
    "FL1" : "5.0",
    "TAU" : "0.00001",'''
    
    rec_str = '''"Receiver" : "comment",
    "SEISMO" : "1",
    "READREC" : "1",
    "REC_FILE" : "./inputs/receiver.dat",
    "REFRECX, REFRECY, REFRECZ" : "0.0 , 0.0 , 0.0",
    "NGEOPH" : "1",
    "REC_ARRAY" : "0", # No array as it is read from the rec file'''
    
    bndry_str = '''"Boundary Conditions" : "comment",
    "FREE_SURF" : "1",
    "ABS_TYPE" : "2",
    "FW" : "15.0",
    "DAMPING" : "8.0",
    "FPML" : "5.0",
    "VPPML" : "3500.0",
    "NPOWER" : "4.0",
    "K_MAX_CPML" : "1.0",
    "BOUNDARY" : "0",'''
    
    default_strs = '\n \n'.join([chkpt_str,
                                 dom_decomp_str,
                                 fdorder_str,
                                 q_str,
                                 rec_str,
                                 bndry_str,
                                ])
    
    return default_strs

In [4]:
def get_monitor_str(tsnap_params, smgrm_dtfn, expname):
    snap_str = '''"Snapshots" : "comment",
    "SNAP" : "4",
    "TSNAP1" : "%.2e",
    "TSNAP2" : "%.2e",
    "TSNAPINC" : "%.2e",
    "IDX" : "1",
    "IDY" : "1",
    "IDZ" : "1",
    "SNAP_FORMAT" : "3",
    "SNAP_FILE" : "./outputs/snap/%s",
    "SNAP_PLANE" : "2",'''%(tsnap_params[0], tsnap_params[1], tsnap_params[2], expname)
    
    smgrm_str = '''"Seismograms" : "comment",
    "NDT, NDTSHIFT" : "%i, 0",
    "SEIS_FORMAT" : "2",
    "SEIS_FILE" : "./outputs/su/%s",'''%(smgrm_dtfn, expname)
    
    log_str = '''"Monitoring the simulation" : "comment",
    "LOG_FILE" : "./outputs/log/%s.log",
    "LOG" : "1",
    "OUT_SOURCE_WAVELET" : "1",
    "OUT_TIMESTEP_INFO" : "10",
    '''%(expname)
    
    monitor_strs = '\n \n'.join([snap_str,smgrm_str,log_str])
    
    return monitor_strs

In [5]:
def get_time_str(dt, tdur):
    t_str = '''"Time Stepping" : "comment",
    "TIME" : "%.3f",
    "DT" : "%.2e",'''%(tdur, dt)
    
    return t_str

In [6]:
def get_subsurfmod_str(n_xyz, d_xyz, expname):
    mod_str1 = '''"3-D Grid" : "comment",
    "NX" : "%i",
    "NY" : "%i",
    "NZ" : "%i",
    "DX" : "%.2f",  # meters
    "DY" : "%.2f",  # meters
    "DZ" : "%.2f",  # meters
'''%(n_xyz[0],n_xyz[1],n_xyz[2],
     d_xyz[0],d_xyz[1],d_xyz[2],)
    
    mod_str2 = '''"Model" : "comment",
    "READMOD" : "1", # Read from file
    "MFILE" : "./inputs/model/%s",
    "WRITE_MODELFILES" : "2",'''%(expname)

    mod_strs = '\n \n'.join([mod_str1,mod_str2])
    return mod_strs

In [7]:
def get_source_str():
    src_str = '''"Source" : "comment",
    "SOURCE_SHAPE" : "1",  # Ricker
    "SOURCE_TYPE" : "1",  # Explosive
    "SRCREC" : "1",  # Read from file
    "SOURCE_FILE" : "./inputs/sources.dat",
    "RUN_MULTIPLE_SHOTS" : "0",
    "PLANE_WAVE_ANGLE" : "0.0",
    "TS" : "0.05",  # Duration of source-signal
    '''
    
    return src_str

In [8]:
expname = 'simple3L'

# mod params
n_xyz = [200, 200, 350]
d_xyz = [5, 5, 5]

# time params (in seconds!)
dt = 1e-5
tdur = 0.1
tsnap_params = [dt, tdur-dt, dt*10]
smgrm_ndt = 40


# GET THE DIFFERENT STRINGS - this will be updated in the future
default_strs = get_default_strings()
monitor_strs = get_monitor_str(tsnap_params, smgrm_ndt, expname)
t_str = get_time_str(dt, tdur)
mod_strs = get_subsurfmod_str(n_xyz, d_xyz,expname)
src_str = get_source_str()
SOFI_str = combineSOFI_strings(default_strs, monitor_strs,  t_str, mod_strs, src_str)

# WRITE TO JSON FILE
text_file = open("../../FDModelling/Simple3Layers/SOFI3D_%s.json"%expname, "wt")
text_file.writelines(SOFI_str)
text_file.close()

In [9]:
print(SOFI_str)

#-----------------------------------------------------------------
#      JSON PARAMETER FILE FOR SOFI3D
#-----------------------------------------------------------------
# description: example of json input file
# description/name of the model: homogeneous full space (hh.c)
#

 
{
 
"Checkpoints" : "comment",
    "CHECKPTREAD" : "0",
    "CHECKPTWRITE" : "0",
    "CHECKPT_FILE" : "tmp/checkpoint_sofi3D",
 
"Domain Decomposition" : "comment",
    "NPROCX" : "2",
    "NPROCY" : "2",
    "NPROCZ" : "2",
 
"FD order" : "comment",
    "FDORDER" : "4",
    "FDORDER_TIME" : "2",
    "FDCOEFF" : "2",
    "fdcoeff values: Taylor=1, Holberg=2" : "comment",
 
"Q-approximation" : "comment",
    "L" : "0",
    "FREF" : "5.0",
    "FL1" : "5.0",
    "TAU" : "0.00001",
 
"Receiver" : "comment",
    "SEISMO" : "4",
    "READREC" : "1",
    "REC_FILE" : "./inputs/receiver.dat",
    "REFRECX, REFRECY, REFRECZ" : "0.0 , 0.0 , 0.0",
    "NGEOPH" : "1",
    "REC_ARRAY" : "0", # No array as it is read fro

In [11]:
sofimaster = '~/Documents/Projects/MicroseismicModelling/SOFI3D-master/bin/'
num_procs = 8
sofi_param_file = "SOFI3D_%s.json"%expname
shellfilename = "../../FDModelling/Simple3Layers/runSOFI3D_%s.sh"%expname

runscript = write_SOFIrunScript(sofimaster, num_procs, sofi_param_file, shellfilename)

# # WRITE TO JSON FILE
# text_file = open("../../FDModelling/Simple3Layers/runSOFI3D_%s.sh"%expname, "wt")
# text_file.writelines(runscript)
# text_file.close()